In [1]:
! pip install transformers
! pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 14.1 MB/s eta 0:00:00


In [2]:
import math
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import string
import torch
import contractions
import re
import sklearn
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Pre-processing

In [4]:
train = pd.read_json("train.jsonl",lines=True)
dev = pd.read_json("dev.jsonl",lines=True)
train.head()

,id,img,label,text
0,42953,img/42953.png,0,its their character not their color that matters
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...
2,13894,img/13894.png,0,putting bows on your pet
3,37408,img/37408.png,0,i love everything and everybody! except for sq...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h..."


In [5]:
del train['id']
del train['img']
train.head()

,label,text
0,0,its their character not their color that matters
1,0,don't be afraid to love again everyone is not ...
2,0,putting bows on your pet
3,0,i love everything and everybody! except for sq...
4,0,"everybody loves chocolate chip cookies, even h..."


In [6]:
del dev['id']
del dev['img']
dev.head()

,label,text
0,1,white people is this a shooting range
1,1,bravery at its finest
2,1,your order comes to $37.50 and your white priv...
3,1,it is time.. to send these parasites back to t...
4,1,mississippi wind chime


In [7]:
contractions.fix(train['text'][0])

'its their character not their color that matters'

In [8]:
# function to remove the punctuation

def preprocess(df):
  for x in range(len(df)):
    df['text'][x]= contractions.fix(df['text'][x])
  return df


In [9]:
cleaned_train = preprocess(train)
cleaned_train.head()

<ipython-input-8-1124a3b6e0e6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][x]= contractions.fix(df['text'][x])


,label,text
0,0,its their character not their color that matters
1,0,do not be afraid to love again everyone is not...
2,0,putting bows on your pet
3,0,i love everything and everybody! except for sq...
4,0,"everybody loves chocolate chip cookies, even h..."


In [10]:
cleaned_test = preprocess(dev)
cleaned_test.head()

<ipython-input-8-1124a3b6e0e6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][x]= contractions.fix(df['text'][x])


,label,text
0,1,white people is this a shooting range
1,1,bravery at its finest
2,1,your order comes to $37.50 and your white priv...
3,1,it is time.. to send these parasites back to t...
4,1,mississippi wind chime


In [11]:
text_test = cleaned_test.text.values
labels_test = cleaned_test.label.values

In [12]:
# Finding the maximum length
max_len_train = 0

for text in cleaned_train.text.values :
  max_len_train = max(max_len_train, len(text))

print('Max sentence length :', max_len_train)

Max sentence length : 433


In [13]:
# Finding the maximum length
max_len_test = 0

for text in text_test :
  max_len_test = max(max_len_test, len(text))

print('Max sentence length :', max_len_test)

Max sentence length : 209


In [14]:
max_len = 433

## Tokenization

In [15]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

example_text = cleaned_train['text'][0]
bert_input = tokenizer(example_text,padding='max_length', max_length = max_len, 
                       truncation=True, return_tensors="pt")

print(example_text)
print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

its their character not their color that matters
tensor([[ 101, 2049, 2037, 2839, 2025, 2037, 3609, 2008, 5609,  102,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    

Création de la classe Dataset qui permet de mettre les datasets avec le texte tokenisé au bon format pour BERT

In [16]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [x for x in df['label']]
        self.texts = [tokenizer(sentence, padding='max_length', max_length = max_len, truncation=True, return_tensors="pt") for sentence in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

Création de la classe BertClassifier qui permet la classification avec BERT + ajout d'une couche linéaire pour notre classification binaire 

In [17]:
from torch import nn
from transformers import BertForSequenceClassification
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

class BertClassifier(nn.Module):

    def __init__(self):

        super(BertClassifier, self).__init__()

        self.bert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, output_attentions = True, output_hidden_states = False)
    
    def forward(self, input_id, mask):

        bert_output = self.bert(input_ids = input_id, attention_mask = mask)
        
        return bert_output

In [18]:
from sklearn.model_selection import train_test_split

cleaned_train, val = train_test_split(cleaned_train, test_size = 500, random_state=42)

In [19]:
cleaned_train.head()

,label,text
2858,1,"no one: white cops: he is black, so i am assum..."
3290,0,listen up sweetheart and let this sink in. i n...
7100,0,gillbet bink says your news sucks
7789,1,when wwe has a ppv in world war ll germany inc...
7470,1,"trans women are women, trans women are women, ..."


In [20]:
val.head()

,label,text
6244,1,the latest toy has hit the shops it is a talki...
6813,0,show intolerance for other's religions expect ...
7019,1,if there is even one homeless child in america...
5537,1,the look he gives you when you are the new goa...
6307,0,when people are still talking about epstein's ...


In [21]:
text_train = cleaned_train.text.values
labels_train = cleaned_train.label.values

text_val = val.text.values
labels_val = val.label.values

In [22]:
# on calcule le nombre de mêmes haineux présents dans les différents datasets pour le recall
nb_positifs_train = labels_train.sum()
nb_positifs_val = labels_val.sum()
print(nb_positifs_train, nb_positifs_val)

2892 158


In [23]:
from torch.optim import AdamW
from tqdm import tqdm  #  to show a smart progress meter when the model is training


def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)


    train_dataloader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size, shuffle=True)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.BCEWithLogitsLoss(weight = torch.tensor([1, 1.787]))  # 5450/3050 = 1.787 pour pondérer notre dataset d'entraînement qui contient 5450 meme non haineux pour 3050 memes haineux
    optimizer = AdamW(model.parameters(), lr= LR)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            total_recall_train = 0


            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.float().to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask).logits                     # pour obtenir le tensor de classification
                
                #Loss
                label_loss = train_label.unsqueeze(1)
                label_loss = torch.cat([1 - label_loss, label_loss], 1)   # pour avoir les labels sous la meme forme que l'output de bert
                batch_loss = criterion(output, label_loss)
                total_loss_train += batch_loss.item()
                
                #Accuracy
                probs = torch.nn.functional.softmax(output, dim = -1).squeeze(dim=0)   # fonction d'activation pour convertir l'output en proba d'appartenir aux classes 0 ou 1
                class_probs = probs[:, 1]  # on récupère uniquement la colonne avec la proba d'appartenir à la classe 1
                predicted_class_id = (class_probs > 0.5).float()  # seuil de classification
                acc = (predicted_class_id == train_label).float().sum().item()  # on compte le nombre de prédictions valides
                total_acc_train += acc

                #Recall
                vrai_positifs = 0
                for x in range(len(predicted_class_id)) :
                  if (predicted_class_id[x] == train_label[x] == 1) :
                   vrai_positifs += 1
                total_recall_train += vrai_positifs

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0
            total_recall_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.float().to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask).logits

                    label_loss = val_label.unsqueeze(1)
                    label_loss = torch.cat([1 - label_loss, label_loss], 1)
                    batch_loss = criterion(output, label_loss)
                    total_loss_val += batch_loss.item()
                    

                    probs = torch.nn.functional.softmax(output, dim = -1).squeeze(dim=0)
                    class_probs = probs[:, 1]
                    predicted_class_id = (class_probs > 0.5).float()
                    acc = (predicted_class_id == val_label).float().sum().item()
                    total_acc_val += acc
                    

                    vrai_positifs = 0
                    for x in range(len(predicted_class_id)) :
                      if (predicted_class_id[x] == val_label[x] == 1) :
                        vrai_positifs += 1
                    total_recall_val += vrai_positifs
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / math.ceil(len(train_data)/batch_size): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Train Recall: {total_recall_train / nb_positifs_train: .3f} \
                | Val Loss: {total_loss_val / math.ceil(len(val_data)/batch_size): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f} \
                | Val Recall: {total_recall_val / nb_positifs_val: .3f}')
                  
EPOCHS = 3
model = BertClassifier()
LR = 1e-6

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [24]:
batch_size = 16
with tf.device('/device:GPU:0'):
  train(model, cleaned_train, val, LR, EPOCHS)


100%|██████████| 500/500 [09:46<00:00,  1.17s/it]


Epochs: 1 | Train Loss:  0.922                 | Train Accuracy:  0.627                 | Train Recall:  0.048                 | Val Loss:  0.849                 | Val Accuracy:  0.684                 | Val Recall:  0.000


100%|██████████| 500/500 [09:43<00:00,  1.17s/it]


Epochs: 2 | Train Loss:  0.850                 | Train Accuracy:  0.670                 | Train Recall:  0.131                 | Val Loss:  0.759                 | Val Accuracy:  0.752                 | Val Recall:  0.418


100%|██████████| 500/500 [09:43<00:00,  1.17s/it]


Epochs: 3 | Train Loss:  0.777                 | Train Accuracy:  0.730                 | Train Recall:  0.451                 | Val Loss:  0.710                 | Val Accuracy:  0.774                 | Val Recall:  0.513


In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score


# Prédire les probabilités pour les données de test
validation = Dataset(val)
val_dataloader = torch.utils.data.DataLoader(validation, batch_size = 16)


with torch.no_grad():
  y_pred = []
  y_true = []
  for inputs, labels in val_dataloader:
      labels = labels.unsqueeze(1).float().cpu()
      mask = inputs['attention_mask'].cuda()
      input_id = inputs['input_ids'].squeeze(1).cuda()

      outputs = model(input_id, mask).logits

      probs = torch.nn.functional.softmax(outputs, dim = -1).squeeze(dim=0)
      class_probs = probs[:, 1]
      predicted_class_id = (class_probs > 0.5).float().cpu()

      y_pred.extend(predicted_class_id.numpy())
      y_true.extend(labels.numpy())


# Calculer l'AUC-ROC
auc_roc = roc_auc_score(y_true, y_pred)
print("AUC-ROC : {:.4f}".format(auc_roc))

# Calculer la courbe ROC
fpr, tpr, _ = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC
plt.figure(figsize=(10,10))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic curve validation')
plt.legend(loc="lower right")
plt.show()

In [30]:
!pip install bertviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.8 MB/s eta 0:00:00


In [31]:
from bertviz import head_view

In [47]:
# Attention vizualisation

sentence1 = "do not be afraid to love again everyone is not like your ex"
sentence2 = "You can not be racist if there is no other race"
inputs = tokenizer.encode_plus(sentence2, return_tensors='pt', add_special_tokens=True)
masks = inputs['attention_mask'].cuda()
input_ids = inputs['input_ids'].cuda()
attention = model(input_ids, masks)[-1]   # tuple contenant les 12 tensors (pour les 12 couches) de taille [1, 12, 13, 13]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)

In [87]:
output = model(input_ids, masks).logits
probs = torch.nn.functional.softmax(output, dim = -1).squeeze(dim=0)
print(probs)

tensor([0.2989, 0.7011], device='cuda:0', grad_fn=<SqueezeBackward1>)


In [32]:
head_view(attention, tokens)

<IPython.core.display.Javascript object>

In [81]:
#print(attention)
last_layer = attention[-1] # last layer  [1, 12, 13, 13]
last_layer = last_layer[-1] # last layer [12,13,13]
print(last_layer)

last_head = last_layer[-1].detach().cpu()  # matrice de score d'attention [13,13] pour la dernière tête de la dernière couche
last_head = last_head.numpy()
print(last_head)
attention_CLS12 = last_head[:,0]
attention_CLS11 = last_layer[-2].detach().cpu().numpy()[:,0]
attention_CLS10 = last_layer[-3].detach().cpu().numpy()[:,0]
attention_CLS9 = last_layer[-4].detach().cpu().numpy()[:,0]
attention_CLS8 = last_layer[-5].detach().cpu().numpy()[:,0]
attention_CLS7 = last_layer[-6].detach().cpu().numpy()[:,0]
attention_CLS6 = last_layer[-7].detach().cpu().numpy()[:,0]
attention_CLS5 = last_layer[-8].detach().cpu().numpy()[:,0]
attention_CLS4 = last_layer[-9].detach().cpu().numpy()[:,0]
attention_CLS3 = last_layer[-10].detach().cpu().numpy()[:,0]
attention_CLS2 = last_layer[-11].detach().cpu().numpy()[:,0]
attention_CLS1 = last_layer[-12].detach().cpu().numpy()[:,0]


data = [attention_CLS1, attention_CLS2, attention_CLS3, attention_CLS4, attention_CLS5, attention_CLS6, attention_CLS7, attention_CLS8, attention_CLS9, attention_CLS10, attention_CLS11,attention_CLS12]

Att_CLS_layer12 = pd.DataFrame(data, columns=['CLS', 'You', 'can', 'not', 'be', 'racist', 'if', 'there', 'is', 'no', 'other', 'race', 'SEP']) # matrice du score d'attention pour la dernière couche pour le token CLS avec les autres tokens pour chaque tête

score_moy_layer12 = Att_CLS_layer12.mean()
score_moy_layer12 = score_moy_layer12.to_frame(name='layer12')

print(score_moy_layer12)

tensor([[[8.6627e-02, 5.5865e-02, 3.5204e-02,  ..., 9.9713e-03,
          1.8227e-01, 2.3063e-01],
         [1.2770e-02, 1.8226e-02, 1.0131e-02,  ..., 2.3402e-03,
          1.7947e-02, 8.8390e-01],
         [1.1025e-02, 1.5208e-02, 1.6443e-02,  ..., 6.4922e-03,
          2.0993e-02, 8.5303e-01],
         ...,
         [2.4374e-02, 3.7823e-03, 2.0989e-03,  ..., 4.0024e-03,
          5.1743e-02, 8.6895e-01],
         [7.8534e-02, 5.9418e-03, 3.2526e-03,  ..., 1.1344e-03,
          2.7583e-02, 8.4601e-01],
         [8.2403e-03, 4.9433e-03, 1.6630e-03,  ..., 1.5641e-03,
          5.6741e-03, 9.5507e-01]],

        [[3.4131e-02, 1.5744e-02, 9.4069e-03,  ..., 5.5032e-02,
          4.1851e-01, 1.8865e-01],
         [2.6652e-02, 2.2751e-03, 1.4150e-03,  ..., 1.0464e-02,
          2.5897e-02, 8.9683e-01],
         [1.5466e-02, 3.7850e-03, 4.2658e-04,  ..., 2.1593e-03,
          3.0805e-03, 9.6721e-01],
         ...,
         [4.2079e-02, 1.2699e-02, 1.1423e-02,  ..., 4.9280e-02,
          3.290

In [85]:
score_moy = score_moy_layer12
for i in range(11) : # on parcoure les couches
  layer = attention[i] 
  layer = layer[-1]
  data = []
  for k in range(12) :  # on parcoure les têtes
    attention_CLS = layer[k].detach().cpu().numpy()[:,0]
    data.append(attention_CLS)
  Att_CLS_layer = pd.DataFrame(data, columns=['CLS', 'You', 'can', 'not', 'be', 'racist', 'if', 'there', 'is', 'no', 'other', 'race', 'SEP']) # matrice du score d'attention pour la couche i pour le token CLS avec les autres tokens pour chaque tête
  score_moy_layer = Att_CLS_layer.mean()
  score_moy_layer = score_moy_layer.to_frame()
  score_moy = pd.concat([score_moy,score_moy_layer], axis=1)

print(score_moy)

         layer12         0         0         0         0         0         0  \
CLS     0.115863  0.572365  0.597946  0.701026  0.470664  0.058951  0.042609   
You     0.028906  0.130010  0.468498  0.254743  0.287566  0.049628  0.071934   
can     0.028801  0.082441  0.471566  0.333844  0.181172  0.042089  0.063109   
not     0.035026  0.109402  0.539087  0.330106  0.201874  0.049295  0.050576   
be      0.032487  0.072160  0.537570  0.317450  0.139807  0.055448  0.046610   
racist  0.058535  0.176525  0.632650  0.358978  0.375499  0.037533  0.037271   
if      0.030585  0.087356  0.484273  0.336611  0.206994  0.051933  0.068565   
there   0.031204  0.082226  0.475423  0.328135  0.209664  0.051248  0.048361   
is      0.019436  0.076680  0.405321  0.287968  0.154409  0.048135  0.037522   
no      0.016110  0.106678  0.419251  0.325825  0.153389  0.036103  0.021831   
other   0.042215  0.073593  0.546094  0.416636  0.185776  0.051975  0.022515   
race    0.073170  0.043694  0.406819  0.

In [90]:
layer12 = score_moy['layer12']
del score_moy['layer12']
score_moy = pd.concat([score_moy,layer12], axis=1)
score_moy.columns = ['layer1', 'layer2', 'layer3', 'layer4', 'layer5', 'layer6', 'layer7', 'layer8', 'layer9', 'layer10', 'layer11', 'layer12']
print(score_moy)

          layer1    layer2    layer3    layer4    layer5    layer6    layer7  \
CLS     0.572365  0.597946  0.701026  0.470664  0.058951  0.042609  0.106785   
You     0.130010  0.468498  0.254743  0.287566  0.049628  0.071934  0.101479   
can     0.082441  0.471566  0.333844  0.181172  0.042089  0.063109  0.082196   
not     0.109402  0.539087  0.330106  0.201874  0.049295  0.050576  0.062209   
be      0.072160  0.537570  0.317450  0.139807  0.055448  0.046610  0.038998   
racist  0.176525  0.632650  0.358978  0.375499  0.037533  0.037271  0.040663   
if      0.087356  0.484273  0.336611  0.206994  0.051933  0.068565  0.038249   
there   0.082226  0.475423  0.328135  0.209664  0.051248  0.048361  0.027134   
is      0.076680  0.405321  0.287968  0.154409  0.048135  0.037522  0.025689   
no      0.106678  0.419251  0.325825  0.153389  0.036103  0.021831  0.014259   
other   0.073593  0.546094  0.416636  0.185776  0.051975  0.022515  0.013279   
race    0.043694  0.406819  0.311356  0.

In [92]:
score = score_moy.mean(axis=1)
score = score.to_frame(name = 'score moy')
print(score)

        score moy
CLS      0.263900
You      0.141180
can      0.120740
not      0.128297
be       0.112124
racist   0.156319
if       0.125416
there    0.115335
is       0.102320
no       0.099716
other    0.120523
race     0.122644
SEP      0.201683


In [93]:
nb_positifs_test = labels_test.sum()

def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size, shuffle=True)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_recall_test = 0
    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.float().to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask).logits

              #Accuracy
              probs = torch.nn.functional.softmax(output, dim = -1).squeeze(dim=0)   # fonction d'activation pour convertir l'output en proba d'appartenir aux classes 0 ou 1
              class_probs = probs[:, 1]  # on récupère uniquement la colonne avec la proba d'appartenir à la classe 1
              predicted_class_id = (class_probs > 0.5).float()  # seuil de classification
              acc = (predicted_class_id == test_label).float().sum().item()  # on compte le nombre de prédictions valides
              total_acc_test += acc

              #Recall
              vrai_positifs = 0
              for x in range(len(predicted_class_id)) :
                if (predicted_class_id[x] == test_label[x] == 1) :
                  vrai_positifs += 1
              total_recall_test += vrai_positifs
    
    print(
        f'Test Accuracy: {total_acc_test / len(test_data): .3f} \
        | Test Recall: {total_recall_test / nb_positifs_test: .3f}')

In [94]:
batch_size = 16

with tf.device('/device:GPU:0'):
  evaluate(model, cleaned_test)

Test Accuracy:  0.544         | Test Recall:  0.180


In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score


# Prédire les probabilités pour les données de test
test = Dataset(cleaned_test)
test_dataloader = torch.utils.data.DataLoader(test, batch_size = 16)


with torch.no_grad():
  y_pred = []
  y_true = []
  for inputs, labels in test_dataloader:
      labels = labels.unsqueeze(1).float().cpu()
      mask = inputs['attention_mask'].cuda()
      input_id = inputs['input_ids'].squeeze(1).cuda()

      outputs = model(input_id, mask).logits

      probs = torch.nn.functional.softmax(outputs, dim = -1).squeeze(dim=0)
      class_probs = probs[:, 1]
      predicted_class_id = (class_probs > 0.5).float().cpu()

      y_pred.extend(predicted_class_id.numpy())
      y_true.extend(labels.numpy())


# Calculer l'AUC-ROC
auc_roc = roc_auc_score(y_true, y_pred)
print("AUC-ROC : {:.4f}".format(auc_roc))

# Calculer la courbe ROC
fpr, tpr, _ = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC
plt.figure(figsize=(10,10))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic curve validation')
plt.legend(loc="lower right")
plt.show()

In [ ]:
torch.save(model.state_dict(), "best_model_sigm.pth")  # si on veut sauvegarder notre modèle avec les poids entraînés

In [ ]:
# Cellule pour regarder un peu comment ça se passe à l'intérieur de la boucle train + à quoi ressemble les input/output/label/mask...
# inutile de faire tourner
from tqdm import tqdm
model = BertClassifier()
train, val = Dataset(cleaned_train), Dataset(cleaned_dev)
batch_size=8

train_dataloader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val, batch_size)

loss = nn.BCEWithLogitsLoss()

with tf.device('/device:GPU:0'):
  for train_input, train_label in tqdm(train_dataloader):
    train_label = train_label.float()
    mask = train_input['attention_mask']
    input_id = train_input['input_ids'].squeeze(1)
    output = model(input_id, mask)
    print(train_label)
    print(output)
    probs = torch.nn.functional.softmax(output, dim = -1).squeeze(dim=0)
    print(probs)
    class_probs = probs[:, 1]
    print(class_probs)
    predicted_class_id = (class_probs > 0.5).float()
    print(predicted_class_id)
    vrai_positifs = 0
    for x in range(batch_size) :
      if (predicted_class_id[x] == train_label[x] == 1) :
        vrai_positifs += 1
    print(vrai_positifs)